<a href="https://colab.research.google.com/github/poo5zan/realized-volatility/blob/dev/Univariate_RV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
RUN_FROM_COLAB = True
if RUN_FROM_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    ROOT_DIR = '/content/drive/My Drive/Research Project/data/'

Mounted at /content/drive


In [ ]:
!pip install darts
!pip install arch

In [14]:
# realized volatility data
import pandas as pd
import os
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import datetime
import time
from arch import arch_model
import matplotlib.pyplot as plt
from numba import jit
from sklearn.metrics import mean_squared_error as mse
import warnings
warnings.filterwarnings('ignore')

In [4]:
if RUN_FROM_COLAB:
    DATA_DIR = '/content/drive/My Drive/Research Project/datasets/'
else:
    DATA_DIR = '/Users/pujanmaharjan/uni adelaide/uofa_research_project/datasets'

def get_all_train_data():
    train = pd.read_csv(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', 'train_time_id_ordered.csv'))
    return train

def get_unique_col_elements(df: pd.DataFrame, col_name: str):
    return list(df[col_name].unique())

In [5]:
df = get_all_train_data()
print('Total data ', df.shape)
df.head()

Total data  (428932, 4)


,stock_id,time_id,target,time_id_order
0,8,4294,0.005047,0
1,47,4294,0.001244,0
2,105,4294,0.002353,0
3,69,4294,0.001616,0
4,111,4294,0.001917,0


In [18]:
TIME_ID_ORDER_COL = 'time_id_order'
def split_by_time_id_order(data, split_size):
    if split_size < 0 and split_size >= 1:
        raise Exception("invalid split size. valid range (0,1)")
    # time_id_order ranges from 0 to 3829
    min_time_id_order = data[TIME_ID_ORDER_COL].min()
    max_time_id_order = data[TIME_ID_ORDER_COL].max()
    time_id_order_ix = int((max_time_id_order - min_time_id_order) * split_size)
    time_id_order_to_split = min_time_id_order + time_id_order_ix
    data_copy = data.copy()

    first_part = data_copy[data_copy[TIME_ID_ORDER_COL] <= time_id_order_to_split]
    second_part = data_copy[data_copy[TIME_ID_ORDER_COL] > time_id_order_to_split]
    return first_part, second_part

def split_into_train_val_test(data, train_size, val_size):
    train_df, val_test_df = split_by_time_id_order(data, train_size)
    val_df, test_df = split_by_time_id_order(val_test_df, val_size)

    return train_df, val_df, test_df

In [19]:
stock_0 = df[df['stock_id'] == 0].reset_index(drop=True)
stock_0

,stock_id,time_id,target,time_id_order
0,0,4294,0.003267,0
1,0,24033,0.002580,1
2,0,5666,0.002051,2
3,0,29740,0.002364,3
4,0,22178,0.001439,4
...,...,...,...,...
3825,0,24913,0.002402,3825
3826,0,32195,0.002311,3826
3827,0,15365,0.002017,3827
3828,0,10890,0.003475,3828


In [23]:
train, val, test = split_into_train_val_test(stock_0, 0.8, 0.5)

In [26]:
train_target = train['target'].to_frame()
print('train_target ', train_target.shape)
val_target = val['target'].to_frame()
print('val_target ', val_target.shape)
test_target = val['target'].to_frame()
print('test_target ', test_target.shape)

train_target  (3064, 1)
val_target  (383, 1)
test_target  (383, 1)


# GARCH

NameError: ignored